In [70]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


### Dataset


In [90]:
# Definindo os padrões de entrada (vértices do cubo)
entradas = np.array([
    [0, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 0, 0],
    [1, 0, 1],
    [1, 1, 0],
    [1, 1, 1]
]).astype(float)

# Vetores de resposta associados a cada padrão de entrada
respostas = np.array([
    [1, -1, -1, -1, -1, -1, -1, -1],
    [-1, 1, -1, -1, -1, -1, -1, -1],
    [-1, -1, 1, -1, -1, -1, -1, -1],
    [-1, -1, -1, 1, -1, -1, -1, -1],
    [-1, -1, -1, -1, 1, -1, -1, -1],
    [-1, -1, -1, -1, -1, 1, -1, -1],
    [-1, -1, -1, -1, -1, -1, 1, -1],
    [-1, -1, -1, -1, -1, -1, -1, 1]
]).astype(float)

In [93]:
# Função para adicionar ruído aos padrões de entrada
def adicionar_ruido(entradas, raio_ruido):
    # Cria uma cópia das entradas para não alterar o array original
    entradas_ruidosas = np.copy(entradas)
    # Adiciona ruído aleatório dentro do raio especificado
    ruido = np.random.uniform(-raio_ruido, raio_ruido, entradas.shape)
    entradas_ruidosas += ruido
    return entradas_ruidosas

# Definindo o raio máximo do ruído
raio_ruido = 0.2

# Criando o conjunto de validação com ruído
conjunto_validacao = adicionar_ruido(entradas, raio_ruido)
conjunto_validacao.shape

(8, 3)

In [73]:
X = entradas.T
Y = respostas.reshape((8, len(respostas)))

print ('The shape of X: ' + str(X.shape))
print ('The shape of Y: ' + str(Y.shape))
print ('I have m = %d training examples!' % (X.shape[1]))


The shape of X: (3, 8)
The shape of Y: (8, 8)
I have m = 8 training examples!


# Perceptron de Rosenblatt

In [74]:
def layer_sizes(X, Y):
    # Size of input layer.
    n_x = X.shape[0]
    # Size of output layer.
    n_y = Y.shape[0]
    
    return (n_x, n_y)


In [75]:
(n_x, n_y) = layer_sizes(X, Y)
print("The size of the input layer is: n_x = " + str(n_x))
print("The size of the output layer is: n_y = " + str(n_y))

The size of the input layer is: n_x = 3
The size of the output layer is: n_y = 8


Temos 8 exemplos de treinamento, com 3 atributos cada.

In [76]:
def initialize_parameters(n_x, n_y):

    # matriz de pesos, inicializada com valores aleatórios e pequenos para evitar a saturação da função de ativação
    W = np.random.randn(n_y,n_x) * 0.01
    b = np.zeros((n_y,1))
    
    assert (W.shape == (n_y, n_x))
    assert (b.shape == (n_y, 1))
    
    parameters = {"W": W,
                  "b": b}
    
    return parameters

In [77]:
def forward_propagation(X, parameters):
  
    W = parameters['W']
    b = parameters['b']
   
    
    # Forward Propagation to calculate Z.
    Z = np.dot(W,X) + b
    Y_hat = Z
    
    #assert(Y_hat.shape == (n_y, X.shape[1]))

    return Y_hat

In [78]:
def compute_cost(Y_hat, Y):
    
    # Number of examples.
    m = Y.shape[1]

    # Compute the cost function.
    cost = np.sum((Y_hat - Y)**2)/(2*m) # Mean Squared Error
    
    return cost

In [79]:
def nn_model(X, Y, num_iterations=10, print_cost=False, learning_rate = 0.01):
    
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[1]
    
    # Initialize parameters
    parameters = initialize_parameters(n_x,n_y)
    
    # Loop
    for i in range(0, num_iterations):
         
        # Forward propagation. Inputs: "X, parameters, n_y". Outputs: "Y_hat".
        Y_hat = forward_propagation(X,parameters)
        
        # Cost function. Inputs: "Y_hat, Y". Outputs: "cost".
        cost = compute_cost(Y_hat,Y)

        # Parameters update:
        # Backward propagation. Inputs: "parameters, X, Y, Y_hat". Outputs: "grads".
        dW = (1/X.shape[1]) * np.dot((Y_hat - Y), X.T)
        db = (1/X.shape[1]) * np.sum(Y_hat - Y, axis=1, keepdims=True)

        # Update rule for each parameter.
        parameters["W"] = parameters["W"] - learning_rate * dW
        parameters["b"] = parameters["b"] - learning_rate * db

    return parameters

In [81]:
parameters = nn_model(X, Y, num_iterations=100, print_cost=True)
print("cost = " + str(compute_cost(forward_propagation(X, parameters), Y)))

cost = 1.6484282182199619


In [98]:
# Função predict para classificação multiclasse
def predict(X, parameters):
    Y_hat = forward_propagation(X, parameters)
    Y_prediction = np.argmax(Y_hat, axis=0)

    return Y_prediction

# Realizando previsões com o modelo treinado
predictions = predict(conjunto_validacao.T, parameters)
print("Predictions:", predictions)

for i in range(8):
    print("Entrada: ", conjunto_validacao[i], "Saída: ", predictions[i] , respostas[predictions[i]])

Predictions: [0 1 2 3 4 5 6 7]
Entrada:  [0.01829261 0.04093529 0.12759377] Saída:  0 [ 1. -1. -1. -1. -1. -1. -1. -1.]
Entrada:  [-0.02180318 -0.00382869  1.08725824] Saída:  1 [-1.  1. -1. -1. -1. -1. -1. -1.]
Entrada:  [-0.07664855  1.03143127  0.11862453] Saída:  2 [-1. -1.  1. -1. -1. -1. -1. -1.]
Entrada:  [0.01099033 0.80024164 1.15350762] Saída:  3 [-1. -1. -1.  1. -1. -1. -1. -1.]
Entrada:  [ 1.02595715  0.04904776 -0.03595533] Saída:  4 [-1. -1. -1. -1.  1. -1. -1. -1.]
Entrada:  [ 0.83802669 -0.15251605  0.87007586] Saída:  5 [-1. -1. -1. -1. -1.  1. -1. -1.]
Entrada:  [ 0.86415394  1.11091083 -0.02273131] Saída:  6 [-1. -1. -1. -1. -1. -1.  1. -1.]
Entrada:  [0.83924986 1.16163806 0.90788903] Saída:  7 [-1. -1. -1. -1. -1. -1. -1.  1.]
